In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import re
from datetime import datetime, timedelta
from datetime import date
import time
import pandas as pd
import os
import pickle
today = date.today()
from selenium.webdriver.chrome.service import Service

webdriver_path = r'C:\Users\ICP0774\OneDrive - ANACOM\Desktop\Outros\geckdriver\chromedriver.exe'
service = Service(webdriver_path)

## MEO

In [2]:
urls = [
    ("https://www.meo.pt/servicos/casa/fibra/pacotes-em-destaque", "Destaques"),
    ('https://www.meo.pt/servicos/casa/fibra/pacotes-com-telemovel', 'M4'),
    ('https://www.meo.pt/servicos/casa/fibra/pacotes-tv-net-voz', 'M3'),
    ('https://www.meo.pt/servicos/casa/fibra/gaming', 'Gaming'),
    ('https://www.meo.pt/servicos/casa/fibra/outros-pacotes', 'Outros Pacotes')
]

chromeOptions = Options()
# chromeOptions.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=chromeOptions)
driver.maximize_window()

def get_data(url, name):
    driver.get(url)
    time.sleep(2)
    if url == urls[0][0]:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.css-1g5s5vy'))).click()

    driver.execute_script("document.body.style.transform='scale(0.9)';")
    driver.execute_script("window.scrollBy(0, 300);")
#     time.sleep(0.5)
    items = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'ul.pack-list > li')))
    data = []
    for item in items:
        pack_title = item.find_elements(By.CSS_SELECTOR, 'div.pack-title')
        prices = []
        promotions = item.find_elements(By.CSS_SELECTOR, 'div.offer-radios div.radio-item input[type="radio"]')
        
        if not promotions: 
            pack_price = item.find_element(By.CSS_SELECTOR, 'div.pack-price').text.strip()
            prices.append([pack_price, price_desc])
        else:
            for promotion in promotions:
#                 driver.execute_script("arguments[0].scrollIntoView();", promotion)
                driver.execute_script("arguments[0].click();", promotion)
#                 promotion.click()
                time.sleep(0.1)
                pack_price = item.find_element(By.CSS_SELECTOR, 'div.pack-price').text.strip()
                price_desc = item.find_element(By.CSS_SELECTOR, 'div.price-desc p.txt-dest').text.strip()
                prices.append([pack_price, price_desc])

        while len(prices) < 3:
            prices.append(['', ''])
            
        velocidade_title = item.find_elements(By.CSS_SELECTOR, 'li.ic-net p.detail-title')
        gigas_title = item.find_elements(By.CSS_SELECTOR, 'li.ic-mobile div.radio-item, li.ic-mobile p.detail-title')
        televisao_title = item.find_elements(By.CSS_SELECTOR, 'li.ic-tv div.radio-item, li.ic-tv p.detail-title')
        cartao_title = item.find_elements(By.CSS_SELECTOR, 'li.ic-card p.detail-title')
        chamadas_title = item.find_elements(By.CSS_SELECTOR, 'li.ic-phone p.detail-desc')

        pack_data = [
            pack_title[0].text.strip() if pack_title else '',
            prices[0],
            prices[1],
            prices[2],
            velocidade_title[0].text.strip() if velocidade_title else '',
            gigas_title[0].text.strip() if gigas_title else '',
            televisao_title[0].text.strip() if televisao_title else '',
            cartao_title[0].text.strip() if cartao_title else '',
            chamadas_title[0].text.strip() if chamadas_title else ''
        ]

        data.append(pack_data)
    return data

if __name__ == '__main__':
    MEO_Data = {}
    
    for url, name in urls:
        data = get_data(url, name)
        MEO_Data[name] = data
    
    print(MEO_Data)
    
    driver.quit()

{'Destaques': [['M3 200 Net Fibra', ['€40,49/mês', 'Inclui 1 mensalidade grátis'], ['€29,49/mês', 'Inclui €11,00 de desconto durante 4 meses'], ['€40,49/mês', 'Oferta de 24 meses de aluguer do Extensor MEO Smart WiFi (€3,99/mês).Vê equipamentos com descontos exclusivos.'], '500/100 Mbps', '', 'MEOBox Android TV 4K', '', 'Redes fixas nacionais 24h + 50 destinos internacionais (noite, 1.000 min)'], ['M3 500 Net Fibra', ['€43,49/mês', 'Inclui 1 mensalidade grátis'], ['€41,49/mês', 'Inclui €2,00 de desconto durante 24 meses'], ['€43,49/mês', 'Recebe coluna portátil bluetooth Bang & Olufsen Beosound Explore no valor €199,99. Vê equipamentos com descontos exclusivos.'], '1.000/200Mbps', '', 'MEOBox Android TV 4K', '', 'Redes fixas nacionais 24h + 50 destinos internacionais (noite, 1.000 min)'], ['M4 200 Net Fibra', ['€60,99/mês', 'Inclui 1 mensalidade grátis'], ['€57,99/mês', 'Inclui €3,00 de desconto durante 24 meses'], ['€60,99/mês', 'Oferta de 24 meses de aluguer do Extensor MEO Smart WiF

In [3]:
# Prepare data for DataFrame
data_for_df_MEO = []
for category, packages in MEO_Data.items():
    for package in packages:
        package_type = package[0]
        price_1, promo_1 = package[1] if package[1] else ['', '']
        price_2, promo_2 = package[2]if package[2] else ['', '']
        price_3, promo_3 = package[3] if package[3] else ['', '']
        speed = package[4]
        gb = package[5]
        tv = package[6]
        card = package[7]
        phone = package[8]
        row = [today, category, package_type, price_1, promo_1, price_2, promo_2, price_3, promo_3, speed, gb, tv, card, phone]
        data_for_df_MEO.append(row)

# Create DataFrame
df_MEO = pd.DataFrame(data_for_df_MEO, columns=['Date', 'Category', 'Package', 'Price_1', 'Promotion_1', 'Price_2', 'Promotion_2', 'Price_3', 'Promotion_3', 'Speed', 'GB', 'TV', 'Card', "Phone"])
df_MEO = df_MEO.assign(Provider = 'MEO')

# Cleaning and formatting the data
df_MEO['Price_1'] = df_MEO['Price_1'].str.replace('€', '').str.replace('/mês', '').str.replace(',', '.')
df_MEO['Price_2'] = df_MEO['Price_2'].str.replace('€', '').str.replace('/mês', '').str.replace(',', '.')
df_MEO['Price_3'] = df_MEO['Price_3'].str.replace('€', '').str.replace('/mês', '').str.replace(',', '.')
df_MEO['Price_1'] = pd.to_numeric(df_MEO['Price_1'])
df_MEO['Price_2'] = pd.to_numeric(df_MEO['Price_2'])
df_MEO['Price_3'] = pd.to_numeric(df_MEO['Price_3'])

df_MEO[['Download_speed', 'Upload_speed']] = df_MEO['Speed'].str.split('/', expand=True)
df_MEO['Download_speed'] = df_MEO['Download_speed'].str.replace(' Mbps', '', regex=False).str.replace('.', '', regex=False)
df_MEO['Upload_speed'] = df_MEO['Upload_speed'].str.replace('Mbps', '', regex=False).str.replace('.', '', regex=False)
df_MEO['Download_speed'] = pd.to_numeric(df_MEO['Download_speed'])
df_MEO['Upload_speed'] = pd.to_numeric(df_MEO['Upload_speed'])
df_MEO = df_MEO.drop(columns=['Speed'])

df_MEO['GB'] = df_MEO['GB'].str.replace('GB', '', regex=False)
df_MEO['GB'] = pd.to_numeric(df_MEO['GB'])

df_MEO['Card'] = df_MEO['Card'].str.replace('cartão', '', regex=False)
df_MEO['Card'] = pd.to_numeric(df_MEO['Card'])
df_MEO

,Date,Category,Package,Price_1,Promotion_1,Price_2,Promotion_2,Price_3,Promotion_3,GB,TV,Card,Phone,Provider,Download_speed,Upload_speed
0,2023-08-31,Destaques,M3 200 Net Fibra,40.49,Inclui 1 mensalidade grátis,29.49,"Inclui €11,00 de desconto durante 4 meses",40.49,Oferta de 24 meses de aluguer do Extensor MEO ...,NaN,MEOBox Android TV 4K,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEO,500.0,100.0
1,2023-08-31,Destaques,M3 500 Net Fibra,43.49,Inclui 1 mensalidade grátis,41.49,"Inclui €2,00 de desconto durante 24 meses",43.49,Recebe coluna portátil bluetooth Bang & Olufse...,NaN,MEOBox Android TV 4K,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEO,1000.0,200.0
2,2023-08-31,Destaques,M4 200 Net Fibra,60.99,Inclui 1 mensalidade grátis,57.99,"Inclui €3,00 de desconto durante 24 meses",60.99,Oferta de 24 meses de aluguer do Extensor MEO ...,10.0,MEOBox Android TV 4K,1.0,Redes fixas nacionais 24h + 50 destinos intern...,MEO,500.0,100.0
3,2023-08-31,M4,M4 200 Net Fibra,60.99,Inclui 1 mensalidade grátis,57.99,"Inclui €3,00 de desconto durante 24 meses",60.99,Oferta de 24 meses de aluguer do Extensor MEO ...,10.0,MEOBox Android TV 4K,1.0,Redes fixas nacionais 24h + 50 destinos intern...,MEO,500.0,100.0
4,2023-08-31,M4,M4 500 Net Fibra,63.99,Inclui 1 mensalidade grátis,60.99,"Inclui €3,00 de desconto durante 24 meses",63.99,Recebe coluna portátil bluetooth Bang & Olufse...,20.0,MEOBox Android TV 4K,1.0,Redes fixas nacionais 24h + 50 destinos intern...,MEO,500.0,100.0
5,2023-08-31,M4,M4 1000 Net Fibra,68.99,Inclui 1 mensalidade grátis,65.99,"Inclui €3,00 de desconto durante 24 meses",68.99,Recebe coluna portátil bluetooth Bang & Olufse...,20.0,MEOBox Android TV 4K,1.0,Redes fixas nacionais 24h + 50 destinos intern...,MEO,1000.0,200.0
6,2023-08-31,M3,M3 200 Net Fibra,40.49,Inclui 1 mensalidade grátis,29.49,"Inclui €11,00 de desconto durante 4 meses",40.49,Oferta de 24 meses de aluguer do Extensor MEO ...,NaN,MEOBox Android TV 4K,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEO,500.0,100.0
7,2023-08-31,M3,M3 500 Net Fibra,43.49,Inclui 1 mensalidade grátis,41.49,"Inclui €2,00 de desconto durante 24 meses",43.49,Recebe coluna portátil bluetooth Bang & Olufse...,NaN,MEOBox Android TV 4K,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEO,1000.0,200.0
8,2023-08-31,M3,M3 1000 Net Fibra,48.49,Inclui 1 mensalidade grátis,46.49,"Inclui €2,00 de desconto durante 24 meses",48.49,Oferta de 24 meses de aluguer do Extensor MEO ...,NaN,MEOBox Android TV 4K,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEO,1000.0,200.0
9,2023-08-31,Gaming,M4 1000 Net Fibra,71.99,Inclui 1 mensalidade grátis,68.99,"Inclui €3,00 de desconto durante 24 meses",71.99,Oferta de 24 meses de aluguer do Extensor MEO ...,10.0,MEOBox Android TV 4K,1.0,Redes fixas nacionais 24h + 50 destinos intern...,MEO,1000.0,400.0


## Vodafone

In [4]:
chromeOptions = Options()
# chromeOptions.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=chromeOptions)
driver.maximize_window()

base_url = "https://www.vodafone.pt/pacotes.html"

def navigate_and_scrape(driver, category_element):
    driver.execute_script("arguments[0].scrollIntoView();", category_element)
    driver.execute_script("arguments[0].click();", category_element) 
    time.sleep(0.5)  # time for the page to load the new content
    category_data = scrape_category(driver)
    return category_data

def scrape_category(driver):
    package_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.display--block > div.vf_experienceFragment'))
    )
    
    num_packages = len(package_elements)
    print(num_packages)
    category_data = []
    for i in range(num_packages):
        package_el = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, f'div.display--block > div.vf_experienceFragment:nth-child({i+1})'))
        )
        print(package_el)
        prices = []
        promotions = []
        
        pack_title = package_el.find_element(By.CSS_SELECTOR, 'div.card-name').text
        print(pack_title)
        package_price = package_el.find_element(By.CSS_SELECTOR, 'div.card-price').text.replace('€', '').replace('/mês', '').replace(',', '.')
        prices.append(package_price)
        print(package_price)
            
        # Get all the parent divs
        promotion_elements = package_el.find_elements(By.CSS_SELECTOR, 'div.binding_vTwo--card--membershipAdvantage--sectionOffers > div')

        if promotion_elements:
            for promotion_el in promotion_elements:
                # Find the child div that actually has the 'selected' class
                child_div = promotion_el.find_element(By.CSS_SELECTOR, 'div')

                # Check if it's displayed and not already selected
                if child_div.is_displayed():
                    promotion_text = child_div.text # get the text of the child div, not the parent
                    # Only click if not already selected
                    if 'selected' not in child_div.get_attribute('class'):
                        child_div.click() # click the child div, not the parent
                        time.sleep(0.1)
                        package_price = package_el.find_element(By.CSS_SELECTOR, 'div.card-price').text.replace('€', '').replace('/mês', '').replace(',', '.')
                        prices.append(package_price)
                        print(package_price)

                    promotions.append(promotion_text)
                    print(promotion_text)

        if package_el.find_elements(By.CSS_SELECTOR, 'div.card-offer-section.tv-section'): 
            pack_tv = package_el.find_element(By.CSS_SELECTOR, 'div.card--offer--content-subtitle.television-subtitle').text
            print(pack_tv)
        else:
            pack_tv = ''

        if package_el.find_elements(By.CSS_SELECTOR, 'div.card-offer-section'): 
            pack_speed = package_el.find_element(By.CSS_SELECTOR, ' div.card--offer--content-subtitle.card--desktop__table-display').text 
            print(pack_speed)
        else:
            pack_speed = ''

        if package_el.find_elements(By.CSS_SELECTOR, 'div.card-offer-section.mobile-offer-section'): 
            pack_phone = package_el.find_element(By.CSS_SELECTOR, 'div.card--offer--content-subtitle.mobile-subtitle').text.split(' ')[0]
            print(pack_phone)
        else:
            pack_phone = ''

        if package_el.find_elements(By.CSS_SELECTOR, 'div.mobile--numbers'):
            pack_card = package_el.find_element(By.CSS_SELECTOR, 'div.mobile--numbers').text
            print(pack_card)
        else:
            pack_card = ''

        if package_el.find_elements(By.CSS_SELECTOR, 'div.card-offer-section.mobile-offer-section'):
            pack_gigas = package_el.find_element(By.CSS_SELECTOR, 'div.card--offer--content-subtitle.mobile-subtitle > span').text.split(' ')[0]
            print(pack_gigas)
        else:
            pack_gigas = ''            

        category_data.append([pack_title, prices, promotions, pack_tv, pack_speed, pack_phone, pack_card, pack_gigas])
    
    return category_data

driver.get(base_url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.confirm'))).click()

category_elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.red-underscore-navbar-item')) 
)

categories = [(el.find_element(By.CSS_SELECTOR, 'span.red-underscore-navbar-title').text, base_url + el.get_attribute('id')) for el in category_elements]


data = []
for category_element in category_elements:
    category_name = category_element.find_element(By.CSS_SELECTOR, 'span.red-underscore-navbar-title').text
    print(f"Scraping category: {category_name}")
    category_data = navigate_and_scrape(driver, category_element)
    category_data = [[category_name] + row for row in category_data]
    data.extend(category_data)

driver.quit()


Scraping category: Tv Net Voz
5
<selenium.webdriver.remote.webelement.WebElement (session="a3457a3c422c0472246c111a51f38409", element="EC32DE315A3C82660BAA04EFCD32A77D_element_65")>
fibra3 Plus
29.5
Desconto online €11 x 4m
38.5
Desconto €2 x 24m
40.5
1 mensalidade
40.5
Super Wi-Fi Extra
VBox 4K
200 / 100 Mbps
<selenium.webdriver.remote.webelement.WebElement (session="a3457a3c422c0472246c111a51f38409", element="EC32DE315A3C82660BAA04EFCD32A77D_element_66")>
fibra3 Gold
30.5
Desconto online €11 x 4m
39.5
Desconto €2 x 24m
41.5
1 mensalidade
41.5
Super Wi-Fi Extra
VBox 4K
500 / 100 Mbps
<selenium.webdriver.remote.webelement.WebElement (session="a3457a3c422c0472246c111a51f38409", element="EC32DE315A3C82660BAA04EFCD32A77D_element_67")>
fibra3 Play
41.5
Desconto €3 x 24m
44.5
1 mensalidade
44.5
TV 32" a €69,9
44.5
Super Wi-Fi Extra
VBoxPro 4K
1 Gbps / 200 Mbps
<selenium.webdriver.remote.webelement.WebElement (session="a3457a3c422c0472246c111a51f38409", element="EC32DE315A3C82660BAA04EFCD32A

In [5]:
# Prepare data for DataFrame
max_len_prices = max(len(row[2]) for row in data)
max_len_promotions = max(len(row[3]) for row in data)

# Prepare data for DataFrame
data_for_df_Vodafone = []
for row in data:
    category, package, prices, promotions, tv, speed, phone, card, gb = row
    new_row = [category, package]
    new_row += prices + [''] * (max_len_prices - len(prices))
    new_row += promotions + [''] * (max_len_promotions - len(promotions))
    new_row += [tv, speed, phone, card, gb]
    data_for_df_Vodafone.append(new_row)

# Create column names
cols = ['Category', 'Package']
cols += [f'Price_{i+1}' for i in range(max_len_prices)]
cols += [f'Promotion_{i+1}' for i in range(max_len_promotions)]
cols += ['TV', 'Speed', 'Phone', 'Card','GB']

# Create DataFrame
df_Vodafone = pd.DataFrame(data_for_df_Vodafone, columns=cols)
df_Vodafone = df_Vodafone.assign(Provider = 'Vodafone')

df_Vodafone.insert(0, 'Date', date.today())

df_Vodafone[['Download_speed', 'Upload_speed']] = df_Vodafone['Speed'].str.split(' / ', expand=True)
df_Vodafone['Upload_speed'] = df_Vodafone['Upload_speed'].str.replace(' Mbps', '')
df_Vodafone['Download_speed'] = df_Vodafone['Download_speed'].str.replace('1 Gbps', '1000')
df_Vodafone['Download_speed'] = pd.to_numeric(df_Vodafone['Download_speed'])
df_Vodafone['Upload_speed'] = pd.to_numeric(df_Vodafone['Upload_speed'])
df_Vodafone = df_Vodafone.drop(columns=['Speed'])

df_Vodafone['GB'] = df_Vodafone['GB'].str.replace('Ilimitado', '999')
df_Vodafone['GB'] = pd.to_numeric(df_Vodafone['GB'])

df_Vodafone['Price_1'] = pd.to_numeric(df_Vodafone['Price_1'])
df_Vodafone['Price_2'] = pd.to_numeric(df_Vodafone['Price_2'])
df_Vodafone['Price_3'] = pd.to_numeric(df_Vodafone['Price_3'])
df_Vodafone['Price_4'] = pd.to_numeric(df_Vodafone['Price_4'])

df_Vodafone

,Date,Category,Package,Price_1,Price_2,Price_3,Price_4,Promotion_1,Promotion_2,Promotion_3,Promotion_4,TV,Phone,Card,GB,Provider,Download_speed,Upload_speed
0,2023-08-31,Tv Net Voz,fibra3 Plus,29.5,38.5,40.5,40.5,Desconto online €11 x 4m,Desconto €2 x 24m,1 mensalidade,Super Wi-Fi Extra,VBox 4K,,,NaN,Vodafone,200,100
1,2023-08-31,Tv Net Voz,fibra3 Gold,30.5,39.5,41.5,41.5,Desconto online €11 x 4m,Desconto €2 x 24m,1 mensalidade,Super Wi-Fi Extra,VBox 4K,,,NaN,Vodafone,500,100
2,2023-08-31,Tv Net Voz,fibra3 Play,41.5,44.5,44.5,44.5,Desconto €3 x 24m,1 mensalidade,"TV 32"" a €69,9",Super Wi-Fi Extra,VBoxPro 4K,,,NaN,Vodafone,1000,200
3,2023-08-31,Tv Net Voz,fibra3 Super Séries,48.5,51.5,51.5,51.5,Desconto €3 x 24m,1 mensalidade,"TV 32"" a €69,9",Super Wi-Fi Extra,Vodafone TV PLAY,,,NaN,Vodafone,1000,200
4,2023-08-31,Tv Net Voz,fibra3 Light,32.9,NaN,NaN,NaN,,,,,Sem Tv Box,,,NaN,Vodafone,30,30
5,2023-08-31,Pacotes com Móvel,fibra4 Plus,58.9,61.9,61.9,NaN,Desconto €3 x 24m,1 mensalidade,Super Wi-Fi Extra,,VBox 4K,1,5 GB,10.0,Vodafone,500,100
6,2023-08-31,Pacotes com Móvel,fibra4 Speed,60.9,63.9,63.9,NaN,Desconto €3 x 24m,1 mensalidade,Super Wi-Fi Extra,,VBox 4K,1,10 GB,20.0,Vodafone,1000,200
7,2023-08-31,Pacotes com Móvel,fibra4 Play,62.9,65.9,65.9,65.9,Desconto €3 x 24m,1 mensalidade,"Oferta TV 32""",Super Wi-Fi Extra,VBoxPro 4K,1,10 GB,20.0,Vodafone,1000,200
8,2023-08-31,Pacotes com Móvel,fibra4 Super Séries,69.9,72.9,72.9,72.9,Desconto €3 x 24m,1 mensalidade,"Oferta TV 32""",Super Wi-Fi Extra,Vodafone TV PLAY,1,10 GB,20.0,Vodafone,1000,200
9,2023-08-31,Pacotes Móvel Ilimitado,fibra4 Infinity,71.9,74.9,74.9,74.9,Desconto €3 x 24m,1 mensalidade,Oferta TV 32”,Super Wi-Fi Extra,Vodafone TV PLAY,1,Velocidade até 10 Mbps,999.0,Vodafone,1000,200


old

In [6]:
# chromeOptions = Options()
# chromeOptions.headless = False
# driver = webdriver.Chrome(options=chromeOptions)
# driver.maximize_window()

# promotion_types = [
#     'div.data-binding-wrapper.offer.multiblock.offers-two-cols.new-box div.data-binding-item',
#     'div.data-binding-wrapper.offer.multiblock.offers-three-cols.block-button-offer.new-box div.data-binding-item',
#     'div.data-binding-wrapper.offer.multiblock.offers-three-cols.new-box div.data-binding-item',
#     'div.data-binding-wrapper.offer.multiblock.offers-three-cols-of-6.block-button-offer.new-box div.data-binding-item'
# ]

# def scrape_category(driver):
#     package_elements = WebDriverWait(driver, 10).until(
#         EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.data-binding-main'))  
#     )

#     category_data = []
#     for package_el in package_elements:
#         prices = []
#         promotions = []
#         pack_title = next((package_el.find_element(By.CSS_SELECTOR, 'div.bind-header-item-1 img').get_attribute("title") for package_el in [package_el] if package_el.find_elements(By.CSS_SELECTOR, 'div.bind-header-item-1 img')), '')
#         pack_title = pack_title.replace("Selo ", "")
        
#         if package_el.find_elements(By.CSS_SELECTOR, 'div.data-binding-price--resume'):
#             package_price = package_el.find_element(By.CSS_SELECTOR, 'div.data-binding-price--resume').text
#             if package_price:
#                 prices.append(package_price)
#                 print(package_price)

#             if package_el.find_elements(By.CSS_SELECTOR, 'div.data-binding-price--resume'):
#                     for promotion_type in promotion_types:
#                         promotion_elements = package_el.find_elements(By.CSS_SELECTOR, promotion_type)
#                         if promotion_elements:
#                             for promotion_el in promotion_elements:
#                                 if promotion_el.is_displayed():
#                                     if 'data-binding-item--active' not in promotion_el.get_attribute('class'):
#                                         promotion_el.click()
#                                         time.sleep(0.1)
#                                     package_price_el = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.data-binding-price--resume')))
#                                     package_price = package_price_el.text
#                                     promotion_text = promotion_el.text 
#                                     if promotion_text and promotion_text not in promotions:
#                                         promotions.append(promotion_text)
#                                         print(promotion_text)
#                                     if package_price and package_price not in prices:
#                                         prices.append(package_price)
#                                         print(package_price)
#                             break
                
#             if package_el.find_element(By.CSS_SELECTOR, 'div.data-binding-price--resume').text:
 
#                 if package_el.find_elements(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.television'):  # check if tv package information exists
#                     pack_tv = package_el.find_element(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.television').text
#                     print(pack_tv)
#                 else:
#                     pack_tv = ''

#                     # first case (singleblock)
#                 if package_el.find_elements(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.internet.singleblock span'):
#                     speed_block = package_el.find_element(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.internet.singleblock')
#                     pack_speed = speed_block.find_element(By.CSS_SELECTOR, 'div.data-binding-item.data-binding-item--active span').text
#                     print(pack_speed)
#                 # second case (multiblock)
#                 elif package_el.find_elements(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.internet.multiblock span'):
#                     speed_block = package_el.find_element(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.internet.multiblock')
#                     pack_speed = speed_block.find_element(By.CSS_SELECTOR, 'div.data-binding-item.data-binding-item--active span').text
#                     print(pack_speed)
#                 else:
#                     pack_speed = ''


#                 if package_el.find_elements(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.phone.singleblock span'):
#                     phone_block = package_el.find_element(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.phone.singleblock')
#                     pack_phone = phone_block.find_element(By.CSS_SELECTOR, 'div.data-binding-item.data-binding-item--active span').text
#                     print(pack_phone)
#                 else:
#                     pack_phone = ''

#                 if package_el.find_elements(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.addnumber.multiblock '):
#                     card_block = package_el.find_element(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.addnumber.multiblock')
#                     pack_card = card_block.find_element(By.CSS_SELECTOR, 'div.data-binding-item.data-binding-item--active span').text
#                     print(pack_card)
#                 else:
#                     pack_card = ''

#                 if package_el.find_elements(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.numberConfigs.multiblock'):
#                     gigas_block = package_el.find_element(By.CSS_SELECTOR, 'div.data-binding-el-wrapper.numberConfigs.multiblock')
#                     if gigas_block.find_elements(By.CSS_SELECTOR, 'div.data-binding-item.data-binding-item--active span'):
#                         pack_gigas = gigas_block.find_element(By.CSS_SELECTOR, 'div.data-binding-item.data-binding-item--active span').text
#                         print(pack_gigas)
#                     elif gigas_block.find_elements(By.CSS_SELECTOR, 'div.data-binding-selection--package-name'):
#                         pack_gigas_text = gigas_block.find_element(By.CSS_SELECTOR, 'div.data-binding-selection--package-name').text
#                         gb_match = re.search(r'\d+\s*GB|Ilimitado', pack_gigas_text, re.IGNORECASE)  # search for a number followed by 'GB' or 'Ilimitado'
#                         pack_gigas = gb_match.group(0) if gb_match else ''  
#                         print(pack_gigas)
#                     else:
#                         pack_gigas = ''
#                 else:
#                     pack_gigas = ''

#                 category_data.append([pack_title, prices, promotions, pack_speed, pack_gigas, pack_tv, pack_card, pack_phone])
        
        
#     return category_data

# driver.get("https://www.vodafone.pt/pacotes.html")
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.confirm'))).click()

# category_elements = WebDriverWait(driver, 10).until(
#     EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#nav-pacotes a')) 
# )


# categories = [(el.text, el.get_attribute('href')) for el in category_elements]


# data = []
# for category_name, category_link in categories:
#     driver.get(category_link)
#     time.sleep(1)
#     category_data = scrape_category(driver)
#     category_data = [[category_name] + row for row in category_data]
#     data.extend(category_data)

# driver.quit()

In [7]:
# # Prepare data for DataFrame
# data_for_df_Vodafone = [[today] + row for row in data]

# # Create DataFrame
# df_Vodafone = pd.DataFrame(data_for_df_Vodafone, columns=['Date', 'Category', 'Package', 'Price', 'Speed', 'GB', 'TV', 'Card', 'Phone'])
# df_Vodafone = df_Vodafone.assign(Provider = 'Vodafone')

# def clean_price(value):
#     euro_indices = [i for i, x in enumerate(value) if x == "€"]
#     backslash_indices = [i for i, x in enumerate(value) if x == "/"]

#     first_price = value[euro_indices[0]+1:backslash_indices[0]].strip().replace(',', '.')
#     try:
#         second_price = value[euro_indices[1]+1:backslash_indices[1]].strip().replace(',', '.')
#     except IndexError: 
#         second_price = 100000

#     return min(float(first_price), float(second_price))

# df_Vodafone['Price'] = df_Vodafone['Price'].apply(clean_price)
# df_Vodafone['TV'] = df_Vodafone['TV'].str.replace('\n', '')

# df_Vodafone['Speed'] = df_Vodafone['Speed'].apply(lambda x: x.split('Mbps', 1)[-1] if x.count('Mbps') > 1 else x)
# df_Vodafone[['Download_speed', 'Upload_speed']] = df_Vodafone['Speed'].str.split('/', expand=True)
# df_Vodafone['Download_speed'] = df_Vodafone['Download_speed'].str.replace(' Gbps', '000', regex=False).str.replace(' Mbps', '', regex=False).str.replace('.', '', regex=False)
# df_Vodafone['Upload_speed'] = df_Vodafone['Upload_speed'].str.replace('Mbps', '', regex=False).str.replace('.', '', regex=False)
# df_Vodafone['Download_speed'] = pd.to_numeric(df_Vodafone['Download_speed'])
# df_Vodafone['Upload_speed'] = pd.to_numeric(df_Vodafone['Upload_speed'])
# df_Vodafone = df_Vodafone.drop(columns=['Speed'])

# df_Vodafone['GB'] = df_Vodafone['GB'].str.replace('GB', '', regex=False)
# df_Vodafone['GB'] = df_Vodafone['GB'].str.replace('Ilimitado', '999', regex=False)
# df_Vodafone['GB'] = pd.to_numeric(df_Vodafone['GB'])

# df_Vodafone['Card'] = df_Vodafone['Card'].str.replace('número', '', regex=False)
# df_Vodafone['Card'] = pd.to_numeric(df_Vodafone['Card'])


# df_Vodafone

## NOS

In [8]:
chromeOptions = Options()
# chromeOptions.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=chromeOptions)
driver.maximize_window()

def scrape_element(package_el, selectors, is_price=False):
    if is_price:
        elements_main = package_el.find_elements(By.CSS_SELECTOR, selectors[0])
        elements_sup = package_el.find_elements(By.CSS_SELECTOR, selectors[1])
        if elements_main and elements_sup:
            return elements_main[0].text + ',' + elements_sup[0].text
        else:
            return ''
    else:
        text_parts = []
        for selector in selectors:
            elements = package_el.find_elements(By.CSS_SELECTOR, selector)
            if elements:
                text_parts.append(elements[0].text)
        return ' '.join(text_parts)

def scrape_category(driver):
    package_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.packs-card'))  
    )

    category_data = []
    for package_el in package_elements:
        data_dict = {}
        title_elements = package_el.find_elements(By.CSS_SELECTOR, 'span.packs-card__name__pack-name.js-gtm-item-name.d-block')
        if title_elements:
            title = title_elements[0].text
            if title: # only add to dict if title is not empty
                data_dict['pack_title'] = title
                data_dict['pack_net'] = scrape_element(package_el, ['span.js-net-value', 'span.js-net-unit'])
                data_dict['pack_tv'] = scrape_element(package_el, ['span.js-update-tv-title', 'span.packs-card__service__wrapper-tv__channels'])
                data_dict['pack_movel'] = scrape_element(package_el, ['span.js-update-movel-value', 'span.js-update-movel-unit'])
                data_dict['pack_cartao'] = scrape_element(package_el, ['span.js-card-number', 'span.js-card-label'])
                
                
                promotion_elements = package_el.find_elements(By.CSS_SELECTOR, 'div.col-12.col-lg-9 > div')
                if promotion_elements:
                    print(f'{title} has promotion')
                    data_dict['promotions'] = []
                    for promotion_container in promotion_elements:
                        individual_promotions = promotion_container.find_elements(By.CSS_SELECTOR, 'div.opt-offer.packs-card__offers__pack-item')
                        for promotion_el in individual_promotions:
                            promotion_el.click()
                            time.sleep(0.1)
                            print(promotion_el.text)
                            package_price = scrape_element(package_el, ['span.js-priceValue.js-mainPriceValue', 'sup.js-supPriceValue'], is_price=True)
                            print(package_price)
                            data_dict['promotions'].append([package_price, promotion_el.text])
                else:
                    print(f'{title} does not have promotion')
                    data_dict['promotions'] = []
                    promotion_el = ''
                    package_price = scrape_element(package_el, ['span.js-priceValue.js-mainPriceValue', 'sup.js-supPriceValue'], is_price=True)
                    data_dict['promotions'].append([package_price, promotion_el])
                    
                category_data.append(data_dict)
    return category_data

driver.get("https://www.nos.pt/particulares/pacotes/Paginas/pacotes-5g.aspx/")
time.sleep(1)
consent_button = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="c-right"]/a[1]'))
)

consent_button.click()

wrapper_div = driver.find_element(By.CSS_SELECTOR, 'div.tablist__list--wrapper')
categories_elements = wrapper_div.find_elements(By.CSS_SELECTOR, 'a.cmp-navigation__item-link')

categories = [(cat.text, cat.get_attribute('href')) for cat in categories_elements]

data = []
for category_name, category_link in categories:
    driver.get(category_link)
    driver.execute_script("document.body.style.transform='scale(0.9)';")
    category_data = scrape_category(driver)
    for item in category_data:
        item['category_name'] = category_name
    data.extend(category_data)

driver.quit()
data

NOS 4 5G has promotion
1 Mensalidade
60,99
Desconto 3€/mês
57,99
NOS 4 5G has promotion
1 Mensalidade
62,99
Desconto 3€/mês
59,99
NOS 4 5G has promotion
1 Mensalidade
63,99
Desconto 3€/mês
60,99
NOS 4 5G has promotion
1 Mensalidade
67,99
Desconto 3€/mês
64,99
NOS 4 5G has promotion
1 Mensalidade
84,99
Desconto 3€/mês
81,99
NOS 3 has promotion
Desconto 11 €/4 meses
29,49
1 Mensalidade
40,49
Desconto 2€/mês
38,49
NOS 3 has promotion
1 Mensalidade
41,49
Desconto 2€/mês
39,49
NOS 3 has promotion
1 Mensalidade
43,49
Desconto 2€/mês
41,49
NOS 3 does not have promotion
NOS 1 does not have promotion
NOS 1 does not have promotion
NOS 1 does not have promotion


[{'pack_title': 'NOS 4 5G',
  'pack_net': '500 Mbps',
  'pack_tv': 'UMA TV 4K 180 canais',
  'pack_movel': '10 GB',
  'pack_cartao': '1 Cartão',
  'promotions': [['60,99', '1 Mensalidade'], ['57,99', 'Desconto 3€/mês']],
  'category_name': 'Net TV Voz + Móvel'},
 {'pack_title': 'NOS 4 5G',
  'pack_net': '500 Mbps',
  'pack_tv': 'UMA TV 4K 180 canais',
  'pack_movel': '20 GB',
  'pack_cartao': '1 Cartão',
  'promotions': [['62,99', '1 Mensalidade'], ['59,99', 'Desconto 3€/mês']],
  'category_name': 'Net TV Voz + Móvel'},
 {'pack_title': 'NOS 4 5G',
  'pack_net': '1 Gbps',
  'pack_tv': 'UMA TV 4K 180 canais',
  'pack_movel': '20 GB',
  'pack_cartao': '1 Cartão',
  'promotions': [['63,99', '1 Mensalidade'], ['60,99', 'Desconto 3€/mês']],
  'category_name': 'Net TV Voz + Móvel'},
 {'pack_title': 'NOS 4 5G',
  'pack_net': '1 Gbps',
  'pack_tv': 'UMA TV 4K 180 canais',
  'pack_movel': '40 GB',
  'pack_cartao': '1 Cartão',
  'promotions': [['67,99', '1 Mensalidade'], ['64,99', 'Desconto 3€/mê

In [9]:
# Add date and category to each dictionary in the list
for d in data:
    d['Date'] = today
    d['Category'] = d.get('category_name', '')
    for i, promo in enumerate(d.get('promotions', []), start=1):
        d[f'Price_{i}'], d[f'Promotion_{i}'] = promo

# Create DataFrame
df_NOS = pd.DataFrame(data)

# Replace ',' with '.' in price columns and convert them to float
for col in df_NOS.columns:
    if col.startswith('Price_'):
        df_NOS[col] = df_NOS[col].str.replace(',', '.').astype(float)

df_NOS['pack_net'] = df_NOS['pack_net'].str.replace(' Gbps', '000', regex=False).str.replace(' Mbps', '', regex=False).str.replace(' Mpbs', '', regex=False).str.replace('.', '', regex=False)
df_NOS['pack_net'] = pd.to_numeric(df_NOS['pack_net'])

df_NOS['pack_movel'] = df_NOS['pack_movel'].str.replace('GB', '', regex=False)
df_NOS['pack_movel'] = df_NOS['pack_movel'].str.replace('Sem Limites Max', '999', regex=False)
df_NOS['pack_movel'] = pd.to_numeric(df_NOS['pack_movel'])

df_NOS['pack_cartao'] = df_NOS['pack_cartao'].str.replace('Cartão', '', regex=False)
df_NOS['pack_cartao'] = df_NOS['pack_cartao'].str.replace('Cartões', '', regex=False)
df_NOS['pack_cartao'] = pd.to_numeric(df_NOS['pack_cartao'])

# Rename columns
df_NOS.rename(columns={'pack_title': 'Package', 'pack_net': 'Download_speed', 'pack_movel': 'GB', 'pack_tv': 'TV', 'pack_cartao': 'Card'}, inplace=True)
df_NOS = df_NOS.assign(Provider = 'NOS')

df_NOS = df_NOS.drop(columns=['promotions'])
df_NOS = df_NOS.drop(columns=['category_name'])

df_NOS

,Package,Download_speed,TV,GB,Card,Date,Category,Price_1,Promotion_1,Price_2,Promotion_2,Price_3,Promotion_3,Provider
0,NOS 4 5G,500,UMA TV 4K 180 canais,10.0,1.0,2023-08-31,Net TV Voz + Móvel,60.99,1 Mensalidade,57.99,Desconto 3€/mês,NaN,NaN,NOS
1,NOS 4 5G,500,UMA TV 4K 180 canais,20.0,1.0,2023-08-31,Net TV Voz + Móvel,62.99,1 Mensalidade,59.99,Desconto 3€/mês,NaN,NaN,NOS
2,NOS 4 5G,1000,UMA TV 4K 180 canais,20.0,1.0,2023-08-31,Net TV Voz + Móvel,63.99,1 Mensalidade,60.99,Desconto 3€/mês,NaN,NaN,NOS
3,NOS 4 5G,1000,UMA TV 4K 180 canais,40.0,1.0,2023-08-31,Net TV Voz + Móvel,67.99,1 Mensalidade,64.99,Desconto 3€/mês,NaN,NaN,NOS
4,NOS 4 5G,1000,UMA TV 4K 180 canais,999.0,1.0,2023-08-31,Net TV Voz + Móvel,84.99,1 Mensalidade,81.99,Desconto 3€/mês,NaN,NaN,NOS
5,NOS 3,200,UMA TV 4K EXCLUSIVO ONLINE:,NaN,NaN,2023-08-31,Net TV Voz,29.49,Desconto 11 €/4 meses,40.49,1 Mensalidade,38.49,Desconto 2€/mês,NOS
6,NOS 3,500,UMA TV 4K 180 Canais,NaN,NaN,2023-08-31,Net TV Voz,41.49,1 Mensalidade,39.49,Desconto 2€/mês,NaN,NaN,NOS
7,NOS 3,1000,UMA TV 4K 180 Canais,NaN,NaN,2023-08-31,Net TV Voz,43.49,1 Mensalidade,41.49,Desconto 2€/mês,NaN,NaN,NOS
8,NOS 3,30,Sem Box,NaN,NaN,2023-08-31,Net TV Voz,32.99,,NaN,NaN,NaN,NaN,NOS
9,NOS 1,30,,NaN,NaN,2023-08-31,Net,24.99,,NaN,NaN,NaN,NaN,NOS


In [10]:
# chromeOptions = Options()
# chromeOptions.headless = False
# driver = webdriver.Chrome(options=chromeOptions)
# driver.maximize_window()

# def scrape_element(package_el, selectors, is_price=False):
#     if is_price:
#         elements_main = package_el.find_elements(By.CSS_SELECTOR, selectors[0])
#         elements_sup = package_el.find_elements(By.CSS_SELECTOR, selectors[1])
#         if elements_main and elements_sup:
#             return elements_main[0].text + ',' + elements_sup[0].text
#         else:
#             return ''
#     else:
#         text_parts = []
#         for selector in selectors:
#             elements = package_el.find_elements(By.CSS_SELECTOR, selector)
#             if elements:
#                 text_parts.append(elements[0].text)
#         return ' '.join(text_parts)



# def scrape_category(driver):
#     package_elements = WebDriverWait(driver, 10).until(
#         EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.packs-card'))  
#     )

#     category_data = []
#     for package_el in package_elements:
#         data_dict = {}
#         title_elements = package_el.find_elements(By.CSS_SELECTOR, 'span.packs-card__name__pack-name.js-gtm-item-name.d-block')
#         if title_elements:
#             title = title_elements[0].text
#             if title: # only add to dict if title is not empty
#                 data_dict['pack_title'] = title
#                 data_dict['pack_net'] = scrape_element(package_el, ['span.js-net-value', 'span.js-net-unit'])
#                 data_dict['pack_tv'] = scrape_element(package_el, ['span.js-update-tv-title', 'span.packs-card__service__wrapper-tv__channels'])
#                 data_dict['pack_movel'] = scrape_element(package_el, ['span.js-update-movel-value', 'span.js-update-movel-unit'])
#                 data_dict['pack_cartao'] = scrape_element(package_el, ['span.js-card-number', 'span.js-card-label'])
#                 data_dict['pack_price'] = scrape_element(package_el, ['span.js-priceValue.js-mainPriceValue', 'sup.js-supPriceValue'], is_price=True)
#                 category_data.append(data_dict)
#     return category_data



# driver.get("https://www.nos.pt/particulares/pacotes/Paginas/pacotes-5g.aspx/")
# time.sleep(1)
# consent_button = WebDriverWait(driver, 5).until(
#     EC.presence_of_element_located((By.XPATH, '//*[@id="c-right"]/a[1]'))
# )

# consent_button.click()

# wrapper_div = driver.find_element(By.CSS_SELECTOR, 'div.tablist__list--wrapper')
# categories_elements = wrapper_div.find_elements(By.CSS_SELECTOR, 'a.cmp-navigation__item-link')

# categories = [(cat.text, cat.get_attribute('href')) for cat in categories_elements]

# data = []
# for category_name, category_link in categories:
#     driver.get(category_link)
#     category_data = scrape_category(driver)
#     for item in category_data:
#         item['category_name'] = category_name
#     data.extend(category_data)

# driver.quit()
# # data

In [11]:
# # Add date and category to each dictionary in the list
# for d in data:
#     d['Date'] = today
#     d['Category'] = d.get('category_name', '')

# # Create DataFrame
# df_NOS = pd.DataFrame.from_records(data, columns=['Date', 'Category', 'pack_title', 'pack_price', 'pack_net', 'pack_movel', 'pack_tv', 'pack_cartao'])

# # Rename columns
# df_NOS.columns = ['Date', 'Category', 'Package', 'Price', 'Download_speed', 'GB', 'TV', 'Card']
# df_NOS = df_NOS.assign(Provider = 'NOS')

# df_NOS['Price'] = df_NOS['Price'].str.replace(',', '.').astype(float)

# df_NOS['Download_speed'] = df_NOS['Download_speed'].str.replace(' Gbps', '000', regex=False).str.replace(' Mbps', '', regex=False).str.replace(' Mpbs', '', regex=False).str.replace('.', '', regex=False)
# df_NOS['Download_speed'] = pd.to_numeric(df_NOS['Download_speed'])

# df_NOS['GB'] = df_NOS['GB'].str.replace('GB', '', regex=False)
# df_NOS['GB'] = df_NOS['GB'].str.replace('Sem Limites Max', '999', regex=False)
# df_NOS['GB'] = pd.to_numeric(df_NOS['GB'])

# df_NOS['Card'] = df_NOS['Card'].str.replace('Cartão', '', regex=False)
# df_NOS['Card'] = pd.to_numeric(df_NOS['Card'])

# df_NOS

## NOWO

In [12]:
urls = [
    ("https://www.nowo.pt/pacotes/tv-net-voz-movel/", "NET+TV+VOZ+MÓVEL"),
    ('https://www.nowo.pt/pacotes/tv-net-voz/', 'NET+TV+VOZ'),
    ('https://www.nowo.pt/pacotes/net/', 'NET'),
    ('https://www.nowo.pt/pacotes/segunda-casa/', 'SEGUNDA CASA')
]
chromeOptions = Options()
# chromeOptions.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=chromeOptions)
driver.maximize_window()

def get_data(url, name):
    global i
    i=0
    driver.get(url)
    time.sleep(1)
    if url == urls[0][0]:  # If this is the first URL in the list
        cookie_acceptance_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'CybotCookiebotDialogBodyButtonAccept')))
        driver.execute_script("arguments[0].click();", cookie_acceptance_button)
        xpaths_gb = [f'//*[@id="app"]/div[1]/div/div[1]/div[4]/div[2]/div/div/div[2]/div[{n}]' for n in range(1, 11)]
        gb_elements = []
        for xpath in xpaths_gb:
            try:
                elements = driver.find_elements(By.XPATH, xpath)
                gb_elements.extend(elements)
            except:
                break
        gb_elements[2].click()
        
    items = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.sm-container')))
    data = []
    for item in items:
        xpaths = [f'//*[@id="0-{n}"]' for n in range(10)]
        speed_elements = []
        for xpath in xpaths:
            try:
                elements = driver.find_elements(By.XPATH, xpath)
                speed_elements.extend(elements)
            except:
                break
        
        ofertas_elements = item.find_elements(By.CSS_SELECTOR, 'div.sm-payment__offers.show-enter-active p.sm-text')
        ofertas = ' '.join(e.text for e in ofertas_elements)
        
        possible_elements = item.find_elements(By.CSS_SELECTOR, 'div.sm-service')

        voz = ''
        minutos = ''

        for element in possible_elements:
            title_element = element.find_element(By.CSS_SELECTOR, 'h3.sm-services__title')
            if title_element:
                title = title_element.text.strip()
                if title == 'VOZ':
                    # Go up to the parent, then find the options
                    voz_elements = element.find_element(By.XPATH, './..').find_elements(By.CSS_SELECTOR, 'div.sm-services div.sm-services__option.sm-services__option--static')
                    voz = voz_elements[0].text.strip() if voz_elements else ''
                elif title == 'MÓVEL':
                    # Go up to the parent, then find the options
                    minutos_elements = element.find_element(By.XPATH, './..').find_elements(By.CSS_SELECTOR, 'div.sm-services div.sm-services__option.sm-services__option--static')
                    minutos = minutos_elements[0].text.strip() if minutos_elements else ''


        if url == urls[0][0]:
            for speed_element in speed_elements:
                driver.execute_script("window.scrollBy(0, -100)")
                speed_element.click()  # Select a speed option
                speed = speed_element.text.strip()
                for i in range(4):  # For each GB option
                    gb_elements[i].click()  # Select a GB option
                    pack_price = item.find_element(By.CSS_SELECTOR, 'div.sm-show-desktop.sm-payment__values p.sm-payment__price')
                    promotion_price = item.find_elements(By.CSS_SELECTOR, 'div.sm-show-desktop.sm-payment__values p.sm-payment__optional')
#                     print(promotion_price[0].text)
#                     print(pack_price.text)
                    gb = gb_elements[i].text.strip()
                    pack_data = [
                        pack_price.text.strip() if pack_price else '',
                        promotion_price[0].text if promotion_price else '',
                        speed,
                        gb,
                        voz,
                        minutos,
                        ofertas]
                    data.append(pack_data)
                    gb_elements[i].click()
        else:
            gb = ''
            for speed_element in speed_elements:
                speed_element.click()
                pack_price = item.find_element(By.CSS_SELECTOR, 'div.sm-show-desktop.sm-payment__values p.sm-payment__price')
                promotion_price = item.find_elements(By.CSS_SELECTOR, 'div.sm-show-desktop.sm-payment__values p.sm-payment__optional.sm-payment__optional--orange')
#                 print(pack_price.text)
                speed = speed_element.text.strip()
                pack_data = [
                    pack_price.text.strip() if pack_price else '',
                    promotion_price[0].text if promotion_price else '',
                    speed,
                    gb,
                    voz,
                    minutos,
                    ofertas]
                data.append(pack_data)

    return data

if __name__ == '__main__':
    NOWO_Data = {}
    
    for url, name in urls:
        data = get_data(url, name)
        NOWO_Data[name] = data
    
#     print(NOWO_Data)
    
    driver.quit()

In [13]:
# Prepare data for DataFrame
data_for_df_NOWO = []
for category, packages in NOWO_Data.items():
    for package in packages:
        row = [today, category] + package
        data_for_df_NOWO.append(row)

# Create DataFrame
df_NOWO = pd.DataFrame(data_for_df_NOWO, columns=['Date', 'Category','Price_1', 'Promotion_1', 'Download_speed', 'GB', 'Card', 'Phone', 'Promotion_2'])
df_NOWO = df_NOWO.assign(Provider = 'NOWO')


df_NOWO['Price_1'] = df_NOWO['Price_1'].str.replace('€', '')
df_NOWO['Price_1'] = df_NOWO['Price_1'].str.replace('/MÊS', '')
df_NOWO['Price_1'] = df_NOWO['Price_1'].str.replace(',', '.').astype(float)
df_NOWO['GB'] = df_NOWO['GB'].str.split('\n').str[0]

df_NOWO['Download_speed'] = df_NOWO['Download_speed'].str.replace(' Gbps', '000', regex=False).str.replace(' Mbps', '', regex=False).str.replace(' Mpbs', '', regex=False).str.replace('.', '', regex=False)
df_NOWO['Download_speed'] = pd.to_numeric(df_NOWO['Download_speed'])

df_NOWO['GB'] = df_NOWO['GB'].str.replace('GB', '', regex=False)
df_NOWO['GB'] = pd.to_numeric(df_NOWO['GB'])

df_NOWO

,Date,Category,Price_1,Promotion_1,Download_speed,GB,Card,Phone,Promotion_2,Provider
0,2023-08-31,NET+TV+VOZ+MÓVEL,22.5,"€37,50 após o 6º mês",120,1.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
1,2023-08-31,NET+TV+VOZ+MÓVEL,25.0,"€40,00 após o 6º mês",120,5.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
2,2023-08-31,NET+TV+VOZ+MÓVEL,25.0,"€45,00 após o 6º mês",120,15.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
3,2023-08-31,NET+TV+VOZ+MÓVEL,40.0,"€55,00 após o 6º mês",120,30.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
4,2023-08-31,NET+TV+VOZ+MÓVEL,25.0,"€40,00 após o 6º mês",250,1.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
5,2023-08-31,NET+TV+VOZ+MÓVEL,27.5,"€42,50 após o 6º mês",250,5.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
6,2023-08-31,NET+TV+VOZ+MÓVEL,27.5,"€42,50 após o 6º mês",250,15.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
7,2023-08-31,NET+TV+VOZ+MÓVEL,42.5,"€57,50 após o 6º mês",250,30.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
8,2023-08-31,NET+TV+VOZ+MÓVEL,27.5,"€42,50 após o 6º mês",500,1.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO
9,2023-08-31,NET+TV+VOZ+MÓVEL,30.0,"€45,00 após o 6º mês",500,5.0,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,Inclui 5.000min/SMS,AS TUAS OFERTAS: 15€ Desconto em NET+TV+VOZ d...,NOWO


## ALL

In [14]:
df_all = pd.concat([df_MEO, df_Vodafone, df_NOS, df_NOWO], ignore_index=True)

cols = list(df_all.columns)
cols.insert(1, cols.pop(cols.index('Provider')))
df_all = df_all[cols]

new_order = ['Date', 'Provider', 'Category', 'Package', 'Price_1', 'Promotion_1', 'Price_2', 'Promotion_2', 'Price_3', 'Promotion_3', 'Price_4', 'Promotion_4', 'Download_speed', 'Upload_speed', 'Card', 'GB', 'Phone', 'TV']
df_all = df_all[new_order]

df_all

,Date,Provider,Category,Package,Price_1,Promotion_1,Price_2,Promotion_2,Price_3,Promotion_3,Price_4,Promotion_4,Download_speed,Upload_speed,Card,GB,Phone,TV
0,2023-08-31,MEO,Destaques,M3 200 Net Fibra,40.49,Inclui 1 mensalidade grátis,29.49,"Inclui €11,00 de desconto durante 4 meses",40.49,Oferta de 24 meses de aluguer do Extensor MEO ...,NaN,NaN,500.0,100.0,NaN,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
1,2023-08-31,MEO,Destaques,M3 500 Net Fibra,43.49,Inclui 1 mensalidade grátis,41.49,"Inclui €2,00 de desconto durante 24 meses",43.49,Recebe coluna portátil bluetooth Bang & Olufse...,NaN,NaN,1000.0,200.0,NaN,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
2,2023-08-31,MEO,Destaques,M4 200 Net Fibra,60.99,Inclui 1 mensalidade grátis,57.99,"Inclui €3,00 de desconto durante 24 meses",60.99,Oferta de 24 meses de aluguer do Extensor MEO ...,NaN,NaN,500.0,100.0,1.0,10.0,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
3,2023-08-31,MEO,M4,M4 200 Net Fibra,60.99,Inclui 1 mensalidade grátis,57.99,"Inclui €3,00 de desconto durante 24 meses",60.99,Oferta de 24 meses de aluguer do Extensor MEO ...,NaN,NaN,500.0,100.0,1.0,10.0,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
4,2023-08-31,MEO,M4,M4 500 Net Fibra,63.99,Inclui 1 mensalidade grátis,60.99,"Inclui €3,00 de desconto durante 24 meses",63.99,Recebe coluna portátil bluetooth Bang & Olufse...,NaN,NaN,500.0,100.0,1.0,20.0,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2023-08-31,NOWO,NET,NaN,25.00,,NaN,AS TUAS OFERTAS: Oferta da 1ª mensalidade,NaN,NaN,NaN,NaN,500.0,NaN,,NaN,,NaN
68,2023-08-31,NOWO,NET,NaN,30.00,,NaN,AS TUAS OFERTAS: Oferta da 1ª mensalidade,NaN,NaN,NaN,NaN,1000.0,NaN,,NaN,,NaN
69,2023-08-31,NOWO,SEGUNDA CASA,NaN,22.50,,NaN,AS TUAS OFERTAS: Oferta da 1ª mensalidade,NaN,NaN,NaN,NaN,250.0,NaN,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,NaN,,NaN
70,2023-08-31,NOWO,SEGUNDA CASA,NaN,25.00,,NaN,AS TUAS OFERTAS: Oferta da 1ª mensalidade,NaN,NaN,NaN,NaN,500.0,NaN,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,NaN,,NaN


In [15]:
# df_all.to_excel(r'C:\Users\ICP0774\OneDrive - ANACOM\Preços_Operadores\Operadores.xlsx', index=False)

In [17]:
df_existing = pd.read_excel(r'C:\Users\ICP0774\OneDrive - ANACOM\Preços_Operadores\Operadores.xlsx')

# Append new data
df_combined = pd.concat([df_existing, df_all], ignore_index=True)

# Write back to Excel
df_combined.to_excel(r'C:\Users\ICP0774\OneDrive - ANACOM\Preços_Operadores\Operadores.xlsx', index=False)

In [18]:
df_combined

,Date,Provider,Category,Package,Price_1,Promotion_1,Price_2,Promotion_2,Price_3,Promotion_3,Price_4,Promotion_4,Download_speed,Upload_speed,Card,GB,Phone,TV
0,2023-08-16 00:00:00,MEO,Destaques,M3,40.49,Inclui 1 mensalidade grátis,29.49,"Inclui €11,00 de desconto durante 4 meses",40.49,Recebe coluna portátil bluetooth Bang & Olufse...,NaN,NaN,200.0,100.0,NaN,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
1,2023-08-16 00:00:00,MEO,Destaques,M3,43.49,Inclui 1 mensalidade grátis,41.49,"Inclui €2,00 de desconto durante 24 meses",43.49,Recebe coluna portátil bluetooth Bang & Olufse...,NaN,NaN,500.0,100.0,NaN,NaN,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
2,2023-08-16 00:00:00,MEO,Destaques,M4,60.99,Inclui 1 mensalidade grátis,57.99,"Inclui €3,00 de desconto durante 24 meses",60.99,Recebe um voucher de desconto de €100 num equi...,NaN,NaN,200.0,100.0,1,5.0,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
3,2023-08-16 00:00:00,MEO,M4,M4,60.99,Inclui 1 mensalidade grátis,57.99,"Inclui €3,00 de desconto durante 24 meses",60.99,Recebe um voucher de desconto de €100 num equi...,NaN,NaN,200.0,100.0,1,5.0,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
4,2023-08-16 00:00:00,MEO,M4,M4,63.99,Inclui 1 mensalidade grátis,60.99,"Inclui €3,00 de desconto durante 24 meses",63.99,Recebe coluna portátil bluetooth Bang & Olufse...,NaN,NaN,500.0,100.0,1,10.0,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,2023-08-31,NOWO,NET,NaN,25.00,,NaN,AS TUAS OFERTAS: Oferta da 1ª mensalidade,NaN,NaN,NaN,NaN,500.0,NaN,,NaN,,NaN
148,2023-08-31,NOWO,NET,NaN,30.00,,NaN,AS TUAS OFERTAS: Oferta da 1ª mensalidade,NaN,NaN,NaN,NaN,1000.0,NaN,,NaN,,NaN
149,2023-08-31,NOWO,SEGUNDA CASA,NaN,22.50,,NaN,AS TUAS OFERTAS: Oferta da 1ª mensalidade,NaN,NaN,NaN,NaN,250.0,NaN,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,NaN,,NaN
150,2023-08-31,NOWO,SEGUNDA CASA,NaN,25.00,,NaN,AS TUAS OFERTAS: Oferta da 1ª mensalidade,NaN,NaN,NaN,NaN,500.0,NaN,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,NaN,,NaN


In [19]:
df_all.to_pickle(f'{today}_table.pkl')

## Data Analysis

In [20]:
df_all['Date'] = pd.to_datetime(df_all['Date'])
df_month = df_all[df_all['Date'].dt.month == 8]

### Lowest Price

In [21]:
# Calculate the average price for each row
price_columns = ['Price_1', 'Price_2', 'Price_3', 'Price_4']
df_month['Avg_Price'] = df_month[price_columns].mean(axis=1)

# Group by Provider and select rows with minimum average price for each provider
df_min_price = df_month.loc[df_month.groupby('Provider')['Avg_Price'].idxmin()]

# Select necessary columns for the final output
selected_columns = ['Date', 'Provider', 'Avg_Price', 'Category', 'Package', 'Download_speed', 'Upload_speed', 'Card', 'GB', 'Phone', 'TV']
df_min_price = df_min_price[selected_columns]
df_min_price.rename(columns={'Avg_Price': 'Price'}, inplace=True)

df_min_price

,Date,Provider,Price,Category,Package,Download_speed,Upload_speed,Card,GB,Phone,TV
14,2023-08-31,MEO,24.99,Outros Pacotes,M1 Net,NaN,NaN,NaN,NaN,,
42,2023-08-31,NOS,24.99,Net,NOS 1,30.0,NaN,NaN,NaN,NaN,
61,2023-08-31,NOWO,15.00,NET+TV+VOZ,NaN,120.0,NaN,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,NaN,,NaN
29,2023-08-31,Vodafone,24.90,Net Voz,fibra2 Plus,30.0,3.0,,NaN,,


Tenho de corrigir ali na MEO. Tenho de arranjar forma de ir busca a speed tmb mesmo sendo opções para carregar. Se conseguir carregar nos dois e ter os preços para os 2 melhor.

### Highest Price

In [22]:
# Calculate the average price for each row
price_columns = ['Price_1', 'Price_2', 'Price_3', 'Price_4']
df_month['Avg_Price'] = df_month[price_columns].mean(axis=1)

# Group by Provider and select rows with minimum average price for each provider
df_max_price = df_month.loc[df_month.groupby('Provider')['Avg_Price'].idxmax()]

# Select necessary columns for the final output
selected_columns = ['Date', 'Provider', 'Avg_Price', 'Category', 'Package', 'Download_speed', 'Upload_speed', 'Card', 'GB', 'Phone', 'TV']
df_max_price = df_max_price[selected_columns]
df_max_price.rename(columns={'Avg_Price': 'Price'}, inplace=True)

df_max_price

,Date,Provider,Price,Category,Package,Download_speed,Upload_speed,Card,GB,Phone,TV
9,2023-08-31,MEO,70.99,Gaming,M4 1000 Net Fibra,1000.0,400.0,1.0,10.0,Redes fixas nacionais 24h + 50 destinos intern...,MEOBox Android TV 4K
37,2023-08-31,NOS,83.49,Net TV Voz + Móvel,NOS 4 5G,1000.0,NaN,1.0,999.0,NaN,UMA TV 4K 180 canais
60,2023-08-31,NOWO,50.00,NET+TV+VOZ+MÓVEL,NaN,1000.0,NaN,9.000min Rede Fixa Nacional e 1.000min Rede Fi...,30.0,Inclui 5.000min/SMS,NaN
26,2023-08-31,Vodafone,84.15,Pacotes Móvel Ilimitado,fibra4 Infinity Gold,1000.0,200.0,Velocidade até 1 Gbps,999.0,1,Vodafone TV PLAY
